# From Demand Forecasts to Capacity Decisions
## Round-03: FedEx Tricolor Capacity Planning

This notebook extends our Round-02 explainable demand forecasting work.
The objective of Round-03 is NOT to re-forecast demand, but to translate
forecasted demand into cost-aware and risk-aware flight capacity decisions
using FedEx Tricolor flight metadata.

Key Principles:
- Forecasts are inputs, not outputs
- Capacity decisions balance cost, risk, and flexibility
- Logic is explainable and planner-friendly


In [2]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)


In [4]:
# Load Round-02 forecast output
forecast_df = pd.read_csv("data/forecasted_demand_output.csv")

# Load FedEx Tricolor flight capacity metadata
capacity_df = pd.read_csv("data/tricolor_flight_capacity.csv")

print("Forecast shape:", forecast_df.shape)
print("Capacity shape:", capacity_df.shape)

forecast_df.head()


Forecast shape: (12000, 14)
Capacity shape: (25, 7)


,customer_id,sku_id,route,time_period,base_demand,promotion_flag,discount_percentage,sentiment_score,review_volume,sustainable_sku_flag,regulation_impact_score,eco_preference_index,forecasted_demand,forecast_confidence
0,CUST_1,SKU_1,DEL-MUM,31-01-2025,71,1,10,0.56,30,0,-0.12,0.06,103,0.87
1,CUST_1,SKU_1,DEL-MUM,28-02-2025,43,0,20,0.94,201,1,0.11,0.61,54,0.86
2,CUST_1,SKU_1,DEL-MUM,31-03-2025,68,0,9,-0.91,197,0,0.03,0.79,60,0.62
3,CUST_1,SKU_1,DEL-MUM,30-04-2025,70,0,19,0.18,140,1,0.14,0.45,76,0.88
4,CUST_1,SKU_1,DEL-MUM,31-05-2025,37,1,27,0.13,274,0,-0.08,0.24,51,0.82


## Input Data Overview

Forecast Data (from Round-02):
- route
- forecasted_demand
- promotion_flag
- forecast_confidence

Flight Capacity Metadata:
- route
- max_capacity
- fixed_operational_cost
- variable_cost_per_unit
- delay_risk_score
- real_time_update_flag

Datasets are joined on route.


In [5]:
planning_df = forecast_df.merge(
    capacity_df,
    on="route",
    how="left"
)

planning_df.head()


,customer_id,sku_id,route,time_period,base_demand,promotion_flag,discount_percentage,sentiment_score,review_volume,sustainable_sku_flag,regulation_impact_score,eco_preference_index,forecasted_demand,forecast_confidence,flight_id,max_capacity,fixed_cost,variable_cost_per_unit,real_time_update_flag,delay_risk_score
0,CUST_1,SKU_1,DEL-MUM,31-01-2025,71,1,10,0.56,30,0,-0.12,0.06,103,0.87,FL_DEL-MUM_1,846,91780,64,0,0.15
1,CUST_1,SKU_1,DEL-MUM,31-01-2025,71,1,10,0.56,30,0,-0.12,0.06,103,0.87,FL_DEL-MUM_2,519,80928,57,0,0.09
2,CUST_1,SKU_1,DEL-MUM,31-01-2025,71,1,10,0.56,30,0,-0.12,0.06,103,0.87,FL_DEL-MUM_3,863,107135,59,0,0.08
3,CUST_1,SKU_1,DEL-MUM,31-01-2025,71,1,10,0.56,30,0,-0.12,0.06,103,0.87,FL_DEL-MUM_4,574,111327,68,1,0.16
4,CUST_1,SKU_1,DEL-MUM,31-01-2025,71,1,10,0.56,30,0,-0.12,0.06,103,0.87,FL_DEL-MUM_5,603,62838,58,0,0.24


In [6]:
def classify_demand_stability(row):
    """
    Rule-based demand stability classification.
    """
    if row["forecast_confidence"] >= 0.7 and row["promotion_flag"] == 0:
        return "Stable"
    return "Volatile"

planning_df["demand_stability"] = planning_df.apply(
    classify_demand_stability, axis=1
)

planning_df[["route", "forecasted_demand", "forecast_confidence", "demand_stability"]].head()


,route,forecasted_demand,forecast_confidence,demand_stability
0,DEL-MUM,103,0.87,Volatile
1,DEL-MUM,103,0.87,Volatile
2,DEL-MUM,103,0.87,Volatile
3,DEL-MUM,103,0.87,Volatile
4,DEL-MUM,103,0.87,Volatile


In [9]:
# ---- Normalize capacity column names to logical names used in decision logic ----

capacity_df = capacity_df.rename(columns={
    "fixed_cost": "fixed_operational_cost"
})

print("Capacity columns after normalization:")
print(capacity_df.columns.tolist())
planning_df = forecast_df.merge(
    capacity_df,
    on="route",
    how="left"
)

planning_df.head()


Capacity columns after normalization:
['flight_id', 'route', 'max_capacity', 'fixed_operational_cost', 'variable_cost_per_unit', 'real_time_update_flag', 'delay_risk_score']


,customer_id,sku_id,route,time_period,base_demand,promotion_flag,discount_percentage,sentiment_score,review_volume,sustainable_sku_flag,regulation_impact_score,eco_preference_index,forecasted_demand,forecast_confidence,flight_id,max_capacity,fixed_operational_cost,variable_cost_per_unit,real_time_update_flag,delay_risk_score
0,CUST_1,SKU_1,DEL-MUM,31-01-2025,71,1,10,0.56,30,0,-0.12,0.06,103,0.87,FL_DEL-MUM_1,846,91780,64,0,0.15
1,CUST_1,SKU_1,DEL-MUM,31-01-2025,71,1,10,0.56,30,0,-0.12,0.06,103,0.87,FL_DEL-MUM_2,519,80928,57,0,0.09
2,CUST_1,SKU_1,DEL-MUM,31-01-2025,71,1,10,0.56,30,0,-0.12,0.06,103,0.87,FL_DEL-MUM_3,863,107135,59,0,0.08
3,CUST_1,SKU_1,DEL-MUM,31-01-2025,71,1,10,0.56,30,0,-0.12,0.06,103,0.87,FL_DEL-MUM_4,574,111327,68,1,0.16
4,CUST_1,SKU_1,DEL-MUM,31-01-2025,71,1,10,0.56,30,0,-0.12,0.06,103,0.87,FL_DEL-MUM_5,603,62838,58,0,0.24


In [10]:
def classify_cost_profile(row):
    """
    Classify cost exposure based on fixed vs variable cost impact.
    """
    fixed_cost = row["fixed_operational_cost"]
    variable_cost_exposure = row["variable_cost_per_unit"] * row["forecasted_demand"]
    
    if fixed_cost >= variable_cost_exposure:
        return "High Fixed Cost Exposure"
    else:
        return "High Variable Cost Exposure"


planning_df["cost_profile"] = planning_df.apply(
    classify_cost_profile, axis=1
)

planning_df[
    [
        "route",
        "fixed_operational_cost",
        "variable_cost_per_unit",
        "forecasted_demand",
        "cost_profile"
    ]
].head()


,route,fixed_operational_cost,variable_cost_per_unit,forecasted_demand,cost_profile
0,DEL-MUM,91780,64,103,High Fixed Cost Exposure
1,DEL-MUM,80928,57,103,High Fixed Cost Exposure
2,DEL-MUM,107135,59,103,High Fixed Cost Exposure
3,DEL-MUM,111327,68,103,High Fixed Cost Exposure
4,DEL-MUM,62838,58,103,High Fixed Cost Exposure


In [11]:
planning_df["delay_risk_flag"] = np.where(
    planning_df["delay_risk_score"] >= 0.3,
    "High Delay Risk",
    "Low Delay Risk"
)

planning_df["flexibility_flag"] = np.where(
    planning_df["real_time_update_flag"] == 1,
    "Flexible",
    "Locked"
)

planning_df[["route", "delay_risk_flag", "flexibility_flag"]].head()


,route,delay_risk_flag,flexibility_flag
0,DEL-MUM,Low Delay Risk,Locked
1,DEL-MUM,Low Delay Risk,Locked
2,DEL-MUM,Low Delay Risk,Locked
3,DEL-MUM,Low Delay Risk,Flexible
4,DEL-MUM,Low Delay Risk,Locked


In [13]:
def classify_demand_stability(row):
    """
    Rule-based demand stability classification.
    """
    if row["forecast_confidence"] >= 0.7 and row["promotion_flag"] == 0:
        return "Stable"
    return "Volatile"

planning_df["demand_stability"] = planning_df.apply(
    classify_demand_stability, axis=1
)

planning_df[["route", "forecasted_demand", "forecast_confidence", "promotion_flag", "demand_stability"]].head()


,route,forecasted_demand,forecast_confidence,promotion_flag,demand_stability
0,DEL-MUM,103,0.87,1,Volatile
1,DEL-MUM,103,0.87,1,Volatile
2,DEL-MUM,103,0.87,1,Volatile
3,DEL-MUM,103,0.87,1,Volatile
4,DEL-MUM,103,0.87,1,Volatile


In [14]:
assert "demand_stability" in planning_df.columns


In [15]:
planning_df["utilization_strategy"] = planning_df.apply(
    decide_utilization_strategy, axis=1
)

planning_df[["route", "demand_stability", "utilization_strategy"]].head()


,route,demand_stability,utilization_strategy
0,DEL-MUM,Volatile,Balanced Allocation
1,DEL-MUM,Volatile,Balanced Allocation
2,DEL-MUM,Volatile,Balanced Allocation
3,DEL-MUM,Volatile,Dynamic Buffer
4,DEL-MUM,Volatile,Balanced Allocation


In [16]:
required_cols = [
    "demand_stability",
    "cost_profile",
    "delay_risk_flag",
    "flexibility_flag"
]

for col in required_cols:
    assert col in planning_df.columns, f"Missing column: {col}"

print("All required decision inputs are present.")


All required decision inputs are present.


In [17]:
def generate_planning_comment(row):
    return (
        f"{row['demand_stability']} demand | "
        f"{row['cost_profile']} | "
        f"{row['delay_risk_flag']} | "
        f"{row['flexibility_flag']} → "
        f"{row['utilization_strategy']}"
    )

planning_df["planning_comment"] = planning_df.apply(
    generate_planning_comment, axis=1
)

planning_df[
    ["route", "forecasted_demand", "utilization_strategy", "planning_comment"]
].head()


,route,forecasted_demand,utilization_strategy,planning_comment
0,DEL-MUM,103,Balanced Allocation,Volatile demand | High Fixed Cost Exposure | L...
1,DEL-MUM,103,Balanced Allocation,Volatile demand | High Fixed Cost Exposure | L...
2,DEL-MUM,103,Balanced Allocation,Volatile demand | High Fixed Cost Exposure | L...
3,DEL-MUM,103,Dynamic Buffer,Volatile demand | High Fixed Cost Exposure | L...
4,DEL-MUM,103,Balanced Allocation,Volatile demand | High Fixed Cost Exposure | L...


In [18]:
final_decisions = planning_df[
    [
        "route",
        "forecasted_demand",
        "max_capacity",
        "demand_stability",
        "cost_profile",
        "delay_risk_flag",
        "flexibility_flag",
        "utilization_strategy",
        "planning_comment"
    ]
]

final_decisions.head()


,route,forecasted_demand,max_capacity,demand_stability,cost_profile,delay_risk_flag,flexibility_flag,utilization_strategy,planning_comment
0,DEL-MUM,103,846,Volatile,High Fixed Cost Exposure,Low Delay Risk,Locked,Balanced Allocation,Volatile demand | High Fixed Cost Exposure | L...
1,DEL-MUM,103,519,Volatile,High Fixed Cost Exposure,Low Delay Risk,Locked,Balanced Allocation,Volatile demand | High Fixed Cost Exposure | L...
2,DEL-MUM,103,863,Volatile,High Fixed Cost Exposure,Low Delay Risk,Locked,Balanced Allocation,Volatile demand | High Fixed Cost Exposure | L...
3,DEL-MUM,103,574,Volatile,High Fixed Cost Exposure,Low Delay Risk,Flexible,Dynamic Buffer,Volatile demand | High Fixed Cost Exposure | L...
4,DEL-MUM,103,603,Volatile,High Fixed Cost Exposure,Low Delay Risk,Locked,Balanced Allocation,Volatile demand | High Fixed Cost Exposure | L...


In [19]:
final_decisions.to_csv("round3_capacity_decisions.csv", index=False)
print("Exported: round3_capacity_decisions.csv")


Exported: round3_capacity_decisions.csv


## Conclusion

This notebook demonstrates how explainable demand forecasts can be
translated into cost-aware and risk-aware capacity planning decisions
for the FedEx Tricolor network.

Rather than optimizing blindly, the approach:
- Manages demand uncertainty
- Balances fixed and variable cost exposure
- Protects service reliability
- Produces explainable, planner-ready decisions

This aligns with real-world FedEx operational planning.
